In [1]:
import os
import tempfile
import smbclient
import sys

current_directory = os.getcwd()
sys.path.append(current_directory)

smb_server_ip = os.environ['SMB_SERVER_IP']
smb_server_port = int(os.environ['SMB_SERVER_PORT'])
smb_username = os.environ['SMB_USERNAME']
smb_password = os.environ['SMB_PASSWORD']
smb_share_name = os.environ['SMB_SHARE_NAME']

# Set the SMB client configuration
smbclient.ClientConfig(username=smb_username, password=smb_password)

def save_dataframe_to_smb_server(df, remote_file_path):
    # Create a temporary file to store the compressed CSV
    with tempfile.NamedTemporaryFile(mode='w+', suffix='.csv.gz', delete=False) as temp_file:
        df.to_csv(temp_file.name, index=False, compression='gzip')
        temp_file_path = temp_file.name

    # Create the full remote path
    remote_path = f"\\\\{smb_server_ip}\\{smb_share_name}\\{remote_file_path}"

    # Upload the temporary compressed CSV file to the SMB share
    with open(temp_file_path, 'rb') as local_file:
        with smbclient.open_file(remote_path, mode='wb') as remote_file:
            remote_file.write(local_file.read())

    # Delete the temporary file
    os.unlink(temp_file_path)
    

def download_dataframe_from_smb_server(remote_file_path):
    # Create the full remote path
    remote_path = f"\\\\{smb_server_ip}\\{smb_share_name}\\{remote_file_path}"

    # Download the compressed CSV file from the SMB share
    with tempfile.NamedTemporaryFile(mode='w+b', suffix='.csv.gz', delete=False) as temp_file:
        with smbclient.open_file(remote_path, mode='rb') as remote_file:
            temp_file.write(remote_file.read())
        temp_file_path = temp_file.name

    # Load the temporary compressed CSV file into a pandas DataFrame
    df = pd.read_csv(temp_file_path, compression='gzip')

    # Delete the temporary file
    os.unlink(temp_file_path)

    return df

KeyError: 'SMB_SERVER_IP'